In [144]:
#!/usr/bin/env python
# coding: utf-8

import ccxt
import pandas as pd
import talib
import time
import datetime
import sqlite3
import sys

TIMEFRAME = '1d'
FIAT='USDT'
EXCHANGE = ccxt.poloniex()

csv = """
1/1/2019,BTC,2.5844556,TREZOR
1/1/2019,BTC,0000.26904512,TREZOR-DAVID
1/1/2019,LTC,0021.22350700,TREZOR
1/1/2019,XMR,0125.00000000,VAIO
1/1/2019,XTZ,400.00000000,GALLEON
1/1/2019,XTZ,0199.00000000,EXODUS
1/1/2019,BTC,0000.24745280,EXODUS
1/1/2019,ETH,0002.38869800,EXODUS
1/1/2019,XMR,0051.83942100,EXODUS
1/1/2019,LTC,0000.54801980,EXODUS
1/1/2019,BTC,0000.00100000,SAMURAI
1/1/2019,BTC,0000.11224188,GREEN
1/1/2019,XMR,0001.13615000,MONERUJO
1/5/2019,XMR,0013.99000000,LOCALMONERO
1/1/2019,USDT,0,cash
""".split()

conn = sqlite3.connect(':memory:')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Diario')
cur.execute("""CREATE TABLE IF NOT EXISTS Diario(date TEXT,
                                                 asset TEXT,
                                                 quantity INTEGER,
                                                 custodian TEXT)""")

for register in csv:
    valores = register.split(',')
    print(valores)
    cur.execute('INSERT INTO Diario (date, asset, quantity, custodian) VALUES (?, ?, ?, ?);', valores)
    
conn.commit()

assets = [x[0] for x in set(cur.execute('SELECT asset from Diario;'))]

portfolio = {asset:
             cur.execute("SELECT sum(quantity) FROM Diario WHERE asset=?",
                         (asset,)).fetchone()[0] for asset in assets}

print(portfolio)
ohlcv = dict()

for asset in assets:
    if asset == 'BTC':
        PAIR = 'BTC/{}'.format(FIAT)
        
    elif asset not in ('BTC',FIAT):
        PAIR = '{}/BTC'.format(asset)
        
    if asset != FIAT:
        try:
            ohlcv[PAIR] = EXCHANGE.fetch_ohlcv(PAIR, TIMEFRAME, since=datetime.datetime(2018,1,1).timestamp())
        except Exception as ex:
            print(ex)
            sys.exit(1)
    
N = len(ohlcv['XTZ/BTC']) # RESTRICCION

prices = pd.DataFrame([(v[0], 1./v[4]) for v in ohlcv['BTC/{}'.format(FIAT)][-N:]], columns=['time',FIAT]).set_index('time')

prices['BTC'] = [1 for v in range(N)]
prices['XMR'] = [v[4] for v in ohlcv['XMR/BTC'][-N:]]
prices['LTC'] = [v[4] for v in ohlcv['LTC/BTC'][-N:]]
prices['ETH'] = [v[4] for v in ohlcv['ETH/BTC'][-N:]]
prices['XTZ'] = [v[4] for v in ohlcv['XTZ/BTC'][-N:]]

prices.index = pd.to_datetime(prices.index, unit='ms')
prices['value_btc'] = pd.Series(portfolio[asset] * prices[asset] for asset in assets).sum()

# weights
prices['BTCw'] = 100*portfolio['BTC']*prices.BTC/prices.value_btc
prices['XMRw'] = 100*portfolio['XMR']*prices.XMR/prices.value_btc
prices['LTCw'] = 100*portfolio['LTC']*prices.LTC/prices.value_btc
prices['ETHw'] = 100*portfolio['ETH']*prices.ETH/prices.value_btc
prices['XTZw'] = 100*portfolio['XTZ']*prices.XTZ/prices.value_btc
prices['USDTw'] = 0


prices['value_fiat'] = round(prices.value_btc / prices.USDT, 2)
prices['24h_change'] = round(prices.value_fiat.pct_change(),2)
prices['1w_change'] = round(prices.value_fiat.pct_change(periods=7),2)

informe_precios = prices[['BTCw','XMRw','LTCw','ETHw','XTZw','value_btc','value_fiat','24h_change','1w_change']].iloc[::-1].head(45)

#informe_portfolio = pd.DataFrame(portfolio.items())


['1/1/2019', 'BTC', '2.5844556', 'TREZOR']
['1/1/2019', 'BTC', '0000.26904512', 'TREZOR-DAVID']
['1/1/2019', 'LTC', '0021.22350700', 'TREZOR']
['1/1/2019', 'XMR', '0125.00000000', 'VAIO']
['1/1/2019', 'XTZ', '400.00000000', 'GALLEON']
['1/1/2019', 'XTZ', '0199.00000000', 'EXODUS']
['1/1/2019', 'BTC', '0000.24745280', 'EXODUS']
['1/1/2019', 'ETH', '0002.38869800', 'EXODUS']
['1/1/2019', 'XMR', '0051.83942100', 'EXODUS']
['1/1/2019', 'LTC', '0000.54801980', 'EXODUS']
['1/1/2019', 'BTC', '0000.00100000', 'SAMURAI']
['1/1/2019', 'BTC', '0000.11224188', 'GREEN']
['1/1/2019', 'XMR', '0001.13615000', 'MONERUJO']
['1/5/2019', 'XMR', '0013.99000000', 'LOCALMONERO']
['1/1/2019', 'USDT', '0', 'cash']
{'XMR': 191.965571, 'ETH': 2.388698, 'BTC': 3.2141954, 'USDT': 0, 'LTC': 21.7715268, 'XTZ': 599}


In [145]:
informe_precios

,BTCw,XMRw,LTCw,ETHw,XTZw,value_btc,value_fiat,24h_change,1w_change
time,,,,,,,,,
2020-04-19,62.414590,30.139983,2.530308,1.179736,3.735382,5.149750,36730.86,-0.01,0.05
2020-04-18,62.575383,29.895477,2.581438,1.202814,3.744889,5.136517,37236.57,0.04,0.07
2020-04-17,62.911832,29.950258,2.558746,1.134916,3.444247,5.109048,35906.25,-0.01,0.03
2020-04-16,62.818662,30.025285,2.546651,1.132559,3.476843,5.116625,36343.23,0.07,-0.03
2020-04-15,62.915375,30.119608,2.523596,1.078703,3.362718,5.108760,33825.85,-0.03,-0.10
2020-04-14,63.274297,29.738603,2.570883,1.084077,3.332140,5.079781,34889.32,0.01,-0.04
2020-04-13,63.575051,29.412891,2.595268,1.080185,3.336605,5.055750,34568.38,-0.01,-0.08
2020-04-12,63.448469,29.463008,2.619291,1.083106,3.386127,5.065836,34969.13,0.00,0.02
2020-04-11,63.306327,29.585747,2.651175,1.083165,3.373586,5.077210,34918.88,0.00,0.01


In [177]:
composition = pd.DataFrame(list(portfolio.items()),columns=['Asset','Qty'])

In [178]:
composition['BTC price'] = composition['Asset'].apply(lambda x: prices[x][-1])
composition['FIAT value'] = composition['Asset'].apply(lambda x: round(prices['value_fiat'][-1]*prices[x+'w'][-1]/100,2))
composition['Weight'] = composition['Asset'].apply(lambda x: round(prices[x+'w'][-1],2))
composition = composition.set_index(keys='Asset')

In [179]:
composition.sort_values('Weight', ascending=False)

,Qty,BTC price,FIAT value,Weight
Asset,,,,
BTC,3.214195,1.000000,22925.42,62.41
XMR,191.965571,0.008085,11070.67,30.14
XTZ,599.000000,0.000321,1372.04,3.74
LTC,21.771527,0.005985,929.40,2.53
ETH,2.388698,0.025434,433.33,1.18
USDT,0.000000,0.000140,0.00,0.00


In [132]:
prices

,USDT,BTC,XMR,LTC,ETH,XTZ,value_btc,BTCw,XMRw,LTCw,ETHw,XTZw,value_fiat,24h_change,1w_change
time,,,,,,,,,,,,,,,
2020-03-24,0.000148,1,0.007000,0.006010,0.020529,0.000264,4.895818,65.651858,27.447902,2.672630,1.001625,3.225985,33034.08,NaN,NaN
2020-03-25,0.000150,1,0.007040,0.005869,0.020357,0.000258,4.896705,65.639967,27.597626,2.609522,0.993052,3.159833,32712.73,-0.01,NaN
2020-03-26,0.000148,1,0.007470,0.005997,0.020546,0.000261,4.984101,64.488968,28.772721,2.619746,0.984695,3.133870,33576.58,0.03,NaN
2020-03-27,0.000157,1,0.007485,0.005976,0.020651,0.000250,4.980222,64.539197,28.851138,2.612296,0.990475,3.006894,31695.22,-0.06,NaN
2020-03-28,0.000160,1,0.007545,0.006224,0.021001,0.000251,4.998511,64.303054,28.976194,2.710840,1.003594,3.006318,31175.27,-0.02,NaN
2020-03-29,0.000170,1,0.007486,0.006311,0.021166,0.000251,4.989511,64.419052,28.801238,2.753810,1.013318,3.012581,29338.22,-0.06,NaN
2020-03-30,0.000156,1,0.007270,0.006089,0.020667,0.000251,4.942102,65.037009,28.238904,2.682239,0.998914,3.042935,31599.39,0.08,NaN
2020-03-31,0.000156,1,0.007428,0.006098,0.020715,0.000251,4.972732,64.636403,28.676327,2.669920,0.995085,3.022264,31880.10,0.01,-0.03
2020-04-01,0.000151,1,0.007284,0.005902,0.020418,0.000244,4.935980,65.117673,28.327870,2.603269,0.988093,2.963096,32794.63,0.03,0.00
